In [1]:
import xml.etree.ElementTree as ET
import os

def parse_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    annotations = []
    
    for obj in root.findall('object'):
        if obj.find('name').text == 'line_segment':
            bbox = obj.find('bndbox')
            xmin = int(bbox.find('xmin').text)
            ymin = int(bbox.find('ymin').text)
            xmax = int(bbox.find('xmax').text)
            ymax = int(bbox.find('ymax').text)
            
            annotations.append((xmin, ymin, xmax, ymax))
            
    return annotations


In [ ]:
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical

def load_dataset(image_dir, annotation_dir):
    images = []
    annotations = []
    
    for filename in os.listdir(image_dir):
        if filename.endswith('.png'):
            img_path = os.path.join(image_dir, filename)
            xml_path = os.path.join(annotation_dir, filename.replace('.png', '.xml'))
            
            # Load image
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (256, 256))
            
            # Parse annotation
            annot = parse_annotation(xml_path)
            
            images.append(img)
            annotations.append(annot)
            
    return np.array(images), annotations


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def build_model():
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    
    # Output layer for bounding box coordinates (x, y, width, height)
    model.add(Dense(4, activation='linear'))
    
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    
    return model


In [ ]:
# Load dataset
image_dir = '/path/to/dataset/images'
annotation_dir = '/path/to/dataset/annotations'
images, annotations = load_dataset(image_dir, annotation_dir)

# Convert annotations to numpy array
annotations = np.array([item for sublist in annotations for item in sublist])

# Normalize images
images = images / 255.0

# Build model
model = build_model()

# Train model
model.fit(images, annotations, epochs=50, batch_size=32, validation_split=0.2)


In [ ]:
def predict_line_segments(model, img_path):
    # Load and preprocess the image
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (256, 256))
    img = np.expand_dims(img, axis=0) / 255.0
    
    # Make prediction
    predictions = model.predict(img)
    
    # Extract bounding box coordinates
    x, y, width, height = predictions[0]
    
    # Mark line segment on the image
    img_marked = cv2.rectangle(img[0], (int(x), int(y)), (int(x + width), int(y + height)), (0, 255, 0), 2)
    
    return img_marked

# Example usage
img_path = '/path/to/new/image.png'
img_marked = predict_line_segments(model, img_path)

# Display the marked image
import matplotlib.pyplot as plt
plt.imshow(img_marked)
plt.axis('off')
plt.show()
